In [10]:
import os
from ccdproc import CCDData
import matplotlib.pyplot as plt
import numpy as np
from astropy.modeling import models, fitting
from astropy import units as u
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky
from astroquery.gaia import Gaia
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from goodman_pipeline.core import ra_dec_to_deg
from astropy.wcs.utils import fit_wcs_from_points
from stsci.stimage import xyxymatch

plt.style.use('dark_background')

for _file in sorted(os.listdir('./data/case_1')):
    print(_file)

ModuleNotFoundError: No module named 'stsci'

In [ ]:
def show_fits(ccd, z1=None, z2=None):
    if z1 is None:
        z1 = np.mean(ccd.data) - 0.5 * np.std(ccd.data)
    else:
        z2 = float(z2)
    if z2 is None:
        z2 = np.median(ccd.data) +  2 * np.std(ccd.data)
    else:
        z2 = float(z2)
        
    fig, ax = plt.subplots(figsize=(16,9))
    ax.imshow(ccd.data, clim=(z1, z2))
    
mask_out_file = 'data/case_1/0347_SN2020uxz-23-10-2020_mask_out.fits'

ccd = CCDData.read(mask_out_file, unit='adu')
show_fits(ccd)


In [ ]:

pixel_scale = Angle(0.15, u.arcsec) # arcseconds
x, y = ccd.data.shape
x_median = np.median(ccd.data[int(x/2. - 0.2 * x): int(x/2. + 0.2 * x),:], axis=0)
y_median = np.median(ccd.data[:, int(x/2. - 0.2 * x): int(x/2. + 0.2 * x)], axis=1)

# fit a box model
x_box_model = models.Box1D(amplitude=np.max(x_median), x_0=int(x/2.), width=int(x/2.))
y_box_model = models.Box1D(amplitude=np.max(y_median), x_0=int(y/2.), width=int(y/2.))

box_fitter = fitting.SimplexLSQFitter()

x_box_fitted = box_fitter(x_box_model, range(x), x_median)
y_box_fitted = box_fitter(y_box_model, range(y), y_median)

fig, (ax1, ax2) = plt.subplots(2, 1 ,figsize=(20,15))
ax1.set_title('X Axis')
ax1.plot(x_median)
ax1.plot(x_box_fitted(range(x)), color='r')
ax2.set_title('Y Axis')
ax2.plot(y_median)
ax2.plot(y_box_fitted(range(y)), color='r')
plt.show()

In [ ]:
print(x_box_fitted)
print(y_box_fitted)

In [ ]:
serial_binning, parallel_binning =  [int(x) for x in ccd.header['CCDSUM'].split()]
print("Serial Binning: {} Paralell Binning: {}".format(serial_binning, parallel_binning))
x_width = x_box_fitted.width.value * (serial_binning * pixel_scale)
y_width = y_box_fitted.width.value * (parallel_binning * pixel_scale)

In [ ]:
mean, median, std = sigma_clipped_stats(ccd.data, sigma=3.0)
print(mean, median, std)
show_fits(ccd)


In [ ]:
daofind = DAOStarFinder(fwhm=3.0, threshold=5. * std)
sources = daofind(ccd.data - median)
for col in sources.colnames:
    sources[col].info.format = '%.8g'
print(sources)

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
source_positions = (sources['xcentroid'], sources['ycentroid'])
print(positions)
apertures = CircularAperture(positions, r=5.)
norm = ImageNormalize(stretch=SqrtStretch())
fig, ax = plt.subplots(figsize=(20,15))
ax.imshow(ccd.data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)

In [ ]:
ra_deg, dec_deg = ra_dec_to_deg(right_ascension=ccd.header['RA'], declination=ccd.header['DEC'])

print(ra_deg, dec_deg)

## Convert pixel coordinates to RA DEC

In [ ]:
def physical_to_sky(ccd, sources, pixel_scale_arcsec=0.15):
    x_length, y_length = ccd.data.shape
    x_center = x_length / 2.
    y_center = y_length / 2.
    x_binning, y_binning = [int(x) for x in ccd.header['CCDSUM'].split()]
    ra_deg, dec_deg = ra_dec_to_deg(right_ascension=ccd.header['RA'], declination=ccd.header['DEC'])
    pixel_scale = Angle(pixel_scale_arcsec, u.arcsec)
    sources_ra = []
    sources_dec = []
    for source in sources.iterrows('xcentroid', 'ycentroid'):
        source_ra = ra_deg * u.deg - (x_center - source[0]) * x_binning * pixel_scale
        sources_ra.append(source_ra)
        source_dec = dec_deg * u.deg - (y_center - source[1]) * y_binning * pixel_scale
        sources_dec.append(source_dec)
#         print(source[0], source[1], source_ra.deg, source_dec.deg)
        
    sources.add_columns(cols=[sources_ra, sources_dec], names=['ra', 'dec'])
    return sources
    
ccd_sky_coords = physical_to_sky(ccd=ccd, sources=sources)
print(ccd_sky_coords)

In [ ]:
coord = SkyCoord(ra=ra_deg, dec=dec_deg, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(x_width.to_string(unit=u.degree, decimal=True), u.deg)
height = u.Quantity(y_width.to_string(unit=u.degree, decimal=True), u.deg)
print(width, height)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
r.pprint()

In [ ]:

gaia_sources = SkyCoord(r['ra'], r['dec'])
# gaia_sources = np.transpose((r['ra'], r['dec']))
print(gaia_sources)
# print(dir(r))
# print(r.pprint_all)
# for _col in r.colnames:
#     print(_col)
# print(r.colnames)

In [ ]:
# print(np.array(sources['xcentroid'].data))
source_positions = (sources['xcentroid'].data, sources['ycentroid'].data)
print(source_positions)

# wcs = fit_wcs_from_points(xy=source_positions, world_coords=gaia_sources)

In [ ]:

gaia_positions = np.transpose((r['ra'], r['dec']))

ccd_positions = np.transpose(ccd_sky_coords)
print(ccd_sky_coords)

field = SkyCoord(ra=ccd_sky_coords['ra'] *u.degree, dec=ccd_sky_coords['dec'] * u.degree)
gaia_catalog = SkyCoord(ra=gaia_sources.ra, dec=gaia_sources.dec)

# idx, d2d, d3d = field.match_to_catalog_sky(gaia_catalog)
idx, d2d, d3d = match_coordinates_sky(field, gaia_catalog)
print(idx)
print(d2d)
print(d3d)
matches = gaia_catalog[idx]
print(matches)
new_ra, new_dec = field.spherical_offsets_to(matches)
print(new_ra.deg)
print(field.ra.deg)
# print((matches.separation_3d(field) == d3d).all())
fix, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,15))
ax1.set_title('GAIA Sources')
ax1.set_xlabel('RA')
ax1.set_ylabel('DEC')
ax1.plot(gaia_sources.ra, gaia_sources.dec, 'o', markersize=12)
ax1.plot(field.ra.deg + new_ra.deg , field.dec.deg + new_dec.deg, '1', color='red', markersize=12)
# ax1.plot(new_ra, new_dec, 'o', color='magenta', markersize=12)
# sub_data = [int(y_box_fitted.x_0.value - y_box_fitted.width.value / 2.):int(y_box_fitted.x_0.value + y_box_fitted.width.value / 2.),int(x_box_fitted.x_0.value - x_box_fitted.width.value / 2.):int(x_box_fitted.x_0.value + x_box_fitted.width.value / 2.)]
ax2.imshow(ccd.data, origin='lower', norm=norm,
           interpolation='nearest')
ax2.set_ylim(int(y_box_fitted.x_0.value - y_box_fitted.width.value / 2.), int(y_box_fitted.x_0.value + y_box_fitted.width.value / 2.))
ax2.set_xlim(int(x_box_fitted.x_0.value - x_box_fitted.width.value / 2.), int(x_box_fitted.x_0.value + x_box_fitted.width.value / 2.))
apertures.plot(color='red', lw=3)
# apertures = CircularAperture(gaia_positions, r=5.)
# apertures.plot(color='white', lw=1.5, alpha=0.5)

In [ ]:
print(gaia_sources)

In [ ]:
print(dir(apertures))
print(apertures.positions)